In [10]:
%pylab inline
from serial.tools.list_ports import comports, grep
import serial

print [j for j in comports()]

dev_name = [j[0] for j in grep("GDS_3254")][0]
dev_name


Populating the interactive namespace from numpy and matplotlib
[('COM4', 'AFG CDC Device (COM4)', 'USB VID:PID=2184:001C SNR=GEQ904866'), ('COM3', 'USB Serial Device (COM3)', 'USB VID:PID=2184:0006 SNR=GEQ110185')]


IndexError: list index out of range

In [ ]:
#Opens port and identifies what is connected to the port
#Should return insturment identity
serial.Serial?
with serial.Serial(dev_name) as s:
    s.write('*idn?\n')
    result = s.readline()
result

In [ ]:
#Opens port and acquires what is saved in memory as string
#Divides this string at hashtag which separates settings from waveform
params = {}
with serial.Serial(dev_name, timeout=1) as s:
    s.write(':ACQ1:MEM?\n')
    def upto_hashtag():
        this_char = s.read(1)
        this_line = ''
        while this_char != '#':          
            this_line += this_char
            this_char = s.read(1)
        return this_line

    #Further divides settings
    preamble = upto_hashtag().split(';')
    
    #Retrieves 'memory' of 25000 from settings
    #Waveform data is 50,000 bytes of binary data (2*mem)
    mem = int(preamble[0].split(',')[1])
    
    #Generates list of parameters in the preamble
    param = dict([tuple(x.split(',')) for x in preamble if len(x.split(',')) == 2])
    print param
    #for x in preamble:
    #    print x.split(',')
    
    #Reads waveform data of 50,000 bytes
    s.read(6)# length of 550000
    data = s.read(50001)
    assert data[-1] == '\n', "data is not followed by newline!"
    data = data[:-1]
    

In [ ]:
dict([('memory','50'), ('something else','20')])

In [ ]:
#Gathers and stores information about waveform data from preamble
source = preamble[4].split(',')[1]
print source
chnum = int(source[2])
print chnum
yscale = preamble[6].split(',')[1]
print yscale
yunits = preamble[5].split(',')[1]
print yunits
xscale = preamble[10].split(',')[1]
print xscale
xunits = preamble[9].split(',')[1]
print xunits

In [ ]:
s.read?

In [ ]:
#Plots waveform data
data_array = fromstring(data,dtype='i2')
print len(data_array)
plot(double(data_array)/double(2**(2*8-1)))
pylab.title(source)
pylab.ylabel(yunits+' ('+yscale+')')
pylab.xlabel(xunits+' ('+xscale+')')

In [ ]:
%pylab inline
from serial.tools.list_ports import comports, grep
import serial

 
print [j for j in comports()]

dev_name = [j[0] for j in grep("GDS_3254")][0]
print dev_name

with serial.Serial(dev_name) as s:
    s.write('*idn?\n')
    result = s.readline()
result

In [ ]:
def capture_channel(ch_num):

    with serial.Serial(dev_name) as s:
        s.write(':ACQ1:MEM?\n')
        def upto_hashtag():
            this_char = s.read(1)
            this_line = ''
            while this_char != '#':          
                this_line += this_char
                this_char = s.read(1)
            return this_line

        preamble = upto_hashtag().split(';')
   
        mem = int(preamble[0].split(',')[1])
    
        for x in range(19):
            list = preamble[x].split(',')
            print list
    
        s.read(6)# length of 550000
        data = s.read(25000)
        wave = fromstring(data,dtype=int)
    
        source = preamble[4].split(',')[1]
        print source
        chnum = int(source[2])
        print chnum
        
        if ch_num == chnum:
            yscale = preamble[6].split(',')[1]
            print yscale
            yunits = preamble[5].split(',')[1]
            print yunits
            xscale = preamble[10].split(',')[1]
            print xscale
            xunits = preamble[9].split(',')[1]
            print xunits
        else: 
            wave = ('error')
        return wave
          
plot(capture_channel(2))

In [ ]:
with serial.Serial(dev_name, timeout=1) as s:
    print s.read(50000)

In [ ]:
preamble = 'one,1;two,2;three,3;four,4;'.split(';')
one = int(preamble[0].split(',')[1])
one

In [ ]:
preamble_str = 'one,1;two,2;three,3;four,4;'
preamble = dict([j.split(',') for j in preamble_str.split(';') if len(j.split(',')) == 2])
preamble['four']

In [ ]:
a = 'memory length,25000'.split(',')
int(a[1])

In [ ]:
with serial.Serial(dev_name) as s:
    s.write(':ACQ1:MEM?\n')
    def read_semicolon_line():
        this_char = s.read(1)
        this_line = ''
        while this_char != ';':          
            this_line += this_char
            this_char = s.read(1)
        return this_line
    
    mylist=[i for i in range(21)]
    for x in range(21):
        mylist[x]=read_semicolon_line().split(',')
    a = mylist[20]
    print a

In [ ]:
with serial.Serial(dev_name) as s:
    s.write(':ACQ1:MEM?\n')
    info = s.readline()
    infolist = info.split('#')
    print infolist
    
preamble = infolist[0].split(';')
mem = int(preamble[0].split(',')[1])
print mem



wave = infolist[1].split('550000')
print "wave:",wave[0]
newwave = fromstring(wave,dtype=float)
plot (newwave)

wave

